# Set seed and logger

In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0,1

env: CUDA_VISIBLE_DEVICES=0,1


In [2]:
import logging
import os
import random
import re
import numpy as np
import torch

def set_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    random.seed(seed)
    
    np.random.seed(seed)
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def set_global_logging_level(level=logging.ERROR, prefixes=("",)):
    """
    Override logging levels of different modules based on their name as a prefix.
    It needs to be invoked after the modules have been loaded so that their loggers have been initialized.

    Args:
        level: desired level. Optional. Default is logging.ERROR
        prefixes: list of one or more str prefixes to match (e.g. ["transformers", "torch"]). Optional.
            Default is `[""]` to match all active loggers.
            The match is a case-sensitive `module_name.startswith(prefix)`
    """
    prefix_re = re.compile(fr'^(?:{"|".join(prefixes)})')
    for name in logging.root.manager.loggerDict:
        if re.match(prefix_re, name):
            logging.getLogger(name).setLevel(level)

In [3]:
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)
# if logger.hasHandlers():
#     logger.handlers.clear()
# console = logging.StreamHandler()
# logger.addHandler(console)

logging.basicConfig(
    format='[%(asctime)s %(levelname)s %(name)s] %(message)s', datefmt='%m/%d %H:%M:%S',
    level=logging.INFO
)
set_global_logging_level(logging.WARNING, ["elasticsearch"])

set_seed(0)

# Load examples

In [4]:
import json

split = 'dev'
samples = []
with open(f"data/hotpot-{split}.tsv") as f:
    for line in f:
        if split == 'test':
            q_id, question = line.strip().split('\t')
            samples.append((q_id, (question,)))
        else:
            q_id, question, answer, sp_facts = line.strip().split('\t')
            sp_facts = json.loads(sp_facts)
            samples.append((q_id, (question, answer, sp_facts)))
print(len(samples))

7405


In [5]:
if split == 'test':
    gold_qas_map = None
    gold_samples = None
    gold_state2action = None
else:
    gold_qas_map = dict(samples)
    with open('data/HotpotQA/hotpot_dev_fullwiki_v1.json') as f:
        gold_samples = json.load(f)
    print(len(gold_samples))
    gold_state2action = dict()
    with open('data/hotpot-step-dev.strict.refined.jsonl') as f:
        for line in f:
            example = json.loads(line)
            gold_state2action[example['_id']] = example['state2action']
    print(len(gold_state2action))

7405
7405


# Load sparse query generator

In [6]:
from retriever import SparseRetriever

sparse_retriever = SparseRetriever('enwiki-20171001-paragraph-5', ['10.60.0.59:9200'], max_retries=3, timeout=30)

[05/14 13:13:21 INFO transformers.file_utils] PyTorch version 1.2.0 available.
[05/14 13:13:29 INFO faiss.loader] Loading faiss with AVX2 support.
[05/14 13:13:29 INFO faiss.loader] Loading faiss.


In [7]:
# %env CLASSPATH=corenlp/*
from drqa.reader import Predictor
import warnings

warnings.filterwarnings("ignore")

# import os
# os.environ['CLASSPATH'] = 'corenlp/*'

qg1 = Predictor(model='ckpts/golden-retriever/hop1.mdl', tokenizer=None, embedding_file='data/glove.840B.300d.txt', num_workers=-1)
qg1.cuda()
qg1.model.network.to(torch.device('cuda:0'))
qg2 = Predictor(model='ckpts/golden-retriever/hop2.mdl', tokenizer=None, embedding_file='data/glove.840B.300d.txt', num_workers=-1)
qg2.cuda()
qg2.model.network.to(torch.device('cuda:0'))

[05/14 13:13:32 INFO drqa.reader.predictor] Initializing model...
[05/14 13:13:32 INFO drqa.reader.model] Loading model ckpts/golden-retriever/hop1.mdl
[05/14 13:13:34 INFO drqa.reader.predictor] Expanding dictionary...
[05/14 13:14:53 INFO drqa.reader.model] Adding 2115762 new words to dictionary...
[05/14 13:14:55 INFO drqa.reader.model] New vocab size: 2195963
[05/14 13:15:02 INFO drqa.reader.model] Loading pre-trained embeddings for 2115762 words from data/glove.840B.300d.txt
[05/14 13:15:12 WARNING root] WARN: Duplicate embedding found for ����������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������
[05/14 13:15:17 WARNING root] WARN: Duplicate embedding found for Kṛṣṇa
[05/14 13:15:49 WARNING root] WARN: Dup

RnnDocReader(
  (embedding): Embedding(2195963, 300, padding_idx=0)
  (qemb_match): SeqAttnMatch(
    (linear): Linear(in_features=300, out_features=300, bias=True)
  )
  (doc_rnn): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(662, 128, bidirectional=True)
      (1): LSTM(256, 128, bidirectional=True)
      (2): LSTM(256, 128, bidirectional=True)
    )
  )
  (question_rnn): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(300, 128, bidirectional=True)
      (1): LSTM(256, 128, bidirectional=True)
      (2): LSTM(256, 128, bidirectional=True)
    )
  )
  (self_attn): LinearSeqAttn(
    (linear): Linear(in_features=768, out_features=1, bias=True)
  )
  (start_attn): BilinearSeqAttn(
    (linear): Linear(in_features=768, out_features=768, bias=True)
  )
  (end_attn): BilinearSeqAttn(
    (linear): Linear(in_features=768, out_features=768, bias=True)
  )
)

# Load dense indexer and dense encoder

In [8]:
import faiss
faiss.omp_set_num_threads(16)

In [9]:
from argparse import Namespace

args = Namespace(**{
    "model_name": "roberta-base",
    "model_path": "ckpts/mdr/q_encoder.pt",
    "index_prefix_path": "data/index/mdr/hotpot-paragraph-q-strict",  # .hnsw
    "index_buffer_size": 50000,
    "max_q_len": 70,
    "max_q_sp_len": 350
})

In [10]:
from transformers import AutoConfig, AutoTokenizer
from mdr.retrieval.models.retriever import RobertaCtxEncoder
from utils.model_utils import load_state

bert_config = AutoConfig.from_pretrained(args.model_name)
dense_tokenizer = AutoTokenizer.from_pretrained(args.model_name)
dense_encoder = RobertaCtxEncoder(bert_config, args)
dense_encoder = load_state(dense_encoder, args.model_path, exact=False)
dense_encoder.to(torch.device('cuda:0'))
dense_encoder.eval()

[05/14 13:23:30 INFO transformers.configuration_utils] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /home/xxx/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
[05/14 13:23:30 INFO transformers.configuration_utils] Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

[05/14 13:23:31 INFO transformers.configuration_utils] loading

RobertaCtxEncoder(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [11]:
from dpr.indexer.faiss_indexers import DenseHNSWFlatIndexer, DenseFlatIndexer

vector_size = bert_config.hidden_size
if args.index_prefix_path.endswith('hnsw'):
    dense_indexer = DenseHNSWFlatIndexer(vector_size, args.index_buffer_size)
else:
    dense_indexer = DenseFlatIndexer(vector_size, args.index_buffer_size)
dense_indexer.deserialize_from(args.index_prefix_path)

[05/14 13:23:48 INFO dpr.indexer.faiss_indexers] Loading index from data/index/mdr/hotpot-paragraph-q-strict
[05/14 13:26:12 INFO dpr.indexer.faiss_indexers] Loaded index of type <class 'faiss.swigfaiss.IndexFlat'> and size 5232080


In [12]:
from retriever import DenseRetriever

dense_retriever = DenseRetriever(dense_indexer, dense_encoder, dense_tokenizer)

# Load corpus

In [13]:
from utils.data_utils import load_corpus_

corpus, title2id = load_corpus_('data/corpus/hotpot-paragraph-5.tsv', for_hotpot=True, require_hyperlinks=True)
print(len(corpus))
print(len(title2id))

[05/14 13:32:40 INFO utils.data_utils] Loaded 5232080 passages from data/corpus/hotpot-paragraph-5.tsv


5232080
5232080


# Load union model

In [14]:
from models.union_model import UnionModel

tokenizer = AutoTokenizer.from_pretrained('google/electra-base-discriminator', use_fast=True,
                                           additional_special_tokens=['[unused0]', '[unused1]',
                                                                      '[unused2]', '[unused3]'])
union_model = UnionModel('google/electra-base-discriminator', max_ans_len=64)

[05/14 13:32:43 INFO transformers.configuration_utils] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/google/electra-base-discriminator/config.json from cache at /home/xxx/.cache/torch/transformers/9236d197566a7f1be2b2151f5afcc5a8e17f31e1e23c52f3cdf2340019986e78.88ba6e8e7d5a7936e86d6f2551fe19c236dc57c24da163907cd0544e9933f6ee
[05/14 13:32:43 INFO transformers.configuration_utils] Model config ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "type_vocab_size": 

In [15]:
init_checkpoint = 'ckpts/td5-exp1-ila.4_electra-base-discriminator_DP0.5_HN2_M2_D2_adamW_SP0.5_B32_LR2.0e-05_WU0.1_E30_S42_04202303_pld-sb0-wo*-cmd10/checkpoint_68000.pt'
device = torch.device("cuda:1")

union_model = load_state(union_model, init_checkpoint)
union_model.to(device)
union_model.eval()

UnionModel(
  (encoder): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

# Env: WikiWorld

In [40]:
import redis
from tqdm.auto import tqdm
from wiki_world import WikiWorld

bm25_redis = redis.Redis(host='10.60.1.79', port=6379, db=3, password='redis4zyc', decode_responses=True)
mdr_redis = redis.Redis(host='10.60.1.79', port=6379, db=4, password='redis4zyc', decode_responses=True)
env = WikiWorld(corpus, title2id, sparse_retriever, dense_retriever, bm25_redis, mdr_redis,
                for_hotpot=True, strict=True, max_ret_size=1000)

# QA agent

In [41]:
import copy
from agent import Agent
from hotpot_evaluate_plus import evaluate, pretty_metrics

faiss.omp_set_num_threads(1)
query_redis = redis.Redis(host='10.60.1.79', port=6379, db=2, password='redis4zyc', decode_responses=True)
agent = Agent(tokenizer, union_model, qg1, qg2, device, env, query_redis,
              actions=("ANSWER", "BM25", "MDR", "LINK"), action_mask=(1, 1, 1, 1),
              memory_size=2, max_seq_len=512, max_q_len=96, max_obs_len=256, strict=True,
              gold_qas_map=gold_qas_map, oracle_belief=False)

In [42]:
predictions = {"answer": dict(), "norm_answer": dict(), "sp": dict(), "_sp": dict(), "spp": dict()}
n_observed = -1
env.reset()
agent.reset()
agent.eval()
q_ids = [sample[0] for sample in samples]
questions = [sample[1][0] for sample in samples]
observations = [None] * len(q_ids)

In [43]:
def force_answer():
    if split != 'test':
        answers, _answers, sp_passages, sp_facts, _sp_facts = agent.force_answer(q_ids, questions)
        tmp_predictions = copy.deepcopy(predictions)
        tmp_predictions['answer'].update(answers)
        tmp_predictions['norm_answer'].update(_answers)
        tmp_predictions['sp'].update(sp_facts)
        tmp_predictions['_sp'].update(_sp_facts)
        tmp_predictions['spp'].update(sp_passages)
        print(pretty_metrics(evaluate(tmp_predictions, gold_samples)))
        del tmp_predictions

In [44]:
MAX_OBS = 10
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

while len(observations) > 0 and n_observed < MAX_OBS:
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in tqdm(zip(q_ids, questions), desc='exe', total=len(q_ids)):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.execute(cmd, q_id, excluded=agent.memory[q_id])  # xxx observed memory
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    print(f"{n_observed}: {len(observations)} remained")
    if n_observed > 0 and split != 'test':
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()


0: 7405 remained


[05/15 10:46:51 INFO agent] 5a77152355429966f1a36c2e: miss SP0 2995506_0 (What Are Little Boys Made Of?) for 0.3236 <= 0.4820 when 232 tokens
[05/15 10:46:55 INFO agent] 5ae305df5542992decbdcdc3: miss SP0 34639380_0 (2012 Louisville Cardinals football team) for 0.3529 <= 0.3849 when 345 tokens
[05/15 10:46:57 INFO agent] 5ab28019554299722f9b4d51: miss SP0 30863419_0 (Son of al Qaeda) for 0.0113 <= 0.8037 when 250 tokens
[05/15 10:46:59 INFO agent] 5a7b46e55542995eb53be8e5: miss SP0 44920366_0 (Spaceware Sky Vision II) for 0.0470 <= 0.4297 when 296 tokens
[05/15 10:47:00 INFO agent] 5ae628f55542995703ce8b32: miss SP0 27362380_0 (Six Violin Sonatas, Op. 5 (Vivaldi)) for 0.1364 <= 0.4814 when 226 tokens
[05/15 10:47:08 INFO agent] 5a7b374c5542995eb53be8cc: miss SP0 4491426_0 (Buttered cat paradox) for 0.0377 <= 0.6577 when 362 tokens
[05/15 10:47:11 INFO agent] 5a8242c155429940e5e1a81d: miss SP0 6513540_0 (Thoughts on Machiavelli) for 0.2711 <= 0.3119 when 233 tokens
[05/15 10:47:20 INFO 


1: 7405 remained
|                  |   EM |   F1 | Prec | Recall |
|------------------|------|------|------|--------|
|           Answer | 0.00 | 0.00 | 0.00 |   0.00 |
|      Norm answer | 0.00 | 0.00 | 0.00 |   0.00 |
| Support sentence | 0.00 | 0.00 | 0.00 |   0.00 |
| Support sentence | 0.00 | 0.00 | 0.00 |   0.00 |
|  Support passage | 0.00 | 0.00 | 0.00 |   0.00 |
|            Joint | 0.00 | 0.00 | 0.00 |   0.00 |
0.999594868332208


[05/15 10:51:36 INFO agent] 5a75e05c55429976ec32bc5f: augment SP ['3434750_0'] into memory
[05/15 10:51:36 WARNING agent] 5a75e05c55429976ec32bc5f: augment false SP 3434750_0 into memory {'10426459_0': 0.9922459721565247, '3434750_0': 1.0}
[05/15 10:51:40 INFO agent] 5a713ea95542994082a3e6e4: miss SP1 23602935_0 (Indigenous peoples of Florida) for 0.0059 <= 0.5659 when 379 tokens
[05/15 10:51:40 INFO agent] 5a80b3a9554299485f5986cc: miss SP1 137582_0 (McLean, Virginia) for 0.1781 <= 0.6269 when 379 tokens
[05/15 10:51:40 INFO agent] 5a80b3a9554299485f5986cc: augment SP ['137582_0'] into memory
[05/15 10:51:46 INFO agent] 5a8739a05542994775f607ab: miss SP0 32700345_0 (New York's 1st State Senate district) for 0.3087 <= 0.3827 when 439 tokens
[05/15 10:51:48 INFO agent] 5ab28019554299722f9b4d51: miss SP0 30863419_0 (Son of al Qaeda) for 0.0113 <= 0.8037 when 419 tokens
[05/15 10:51:48 INFO agent] 5a7ed2c655429930675135e5: miss SP1 39367266_0 (Brooklyn Nine-Nine) for 0.0264 <= 0.5958 when


2: 2261 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 46.33 | 55.62 | 57.47 |  56.60 |
|      Norm answer | 46.12 | 55.39 | 57.28 |  56.32 |
| Support sentence | 43.32 | 60.34 | 62.01 |  60.04 |
| Support sentence | 42.28 | 60.01 | 62.60 |  59.29 |
|  Support passage | 64.32 | 66.58 | 66.58 |  66.58 |
|            Joint | 31.55 | 49.95 | 52.92 |  50.49 |
1.999594868332208


...


8: 788 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 58.89 | 70.71 | 73.13 |  71.91 |
|      Norm answer | 58.37 | 70.15 | 72.62 |  71.28 |
| Support sentence | 52.40 | 75.89 | 78.31 |  75.37 |
| Support sentence | 50.74 | 75.42 | 79.12 |  74.38 |
|  Support passage | 80.72 | 84.38 | 84.38 |  84.38 |
|            Joint | 38.22 | 62.62 | 66.69 |  63.20 |
3.0297096556380825


[05/15 11:07:06 INFO agent] 5ae1f596554299234fd04372: augment SP ['24942471_0'] into memory
[05/15 11:07:06 WARNING agent] 5ae1f596554299234fd04372: augment false SP 24942471_0 into memory {'2246220_0': 0.9069764614105225, '24942471_0': 0.12845579385757447}
[05/15 11:07:08 INFO agent] 5ae6b6065542991bbc976168: remove SP0 23636141_0 (Universal Soldier (franchise)) from memory for 0.9061 -> 0.2042 <= 0.3807 when 295 tokens
[05/15 11:07:08 INFO agent] 5ac00f795542996f0d89cb16: miss SP0 230839_0 (Mickey's Christmas Carol) for 0.3208 <= 0.4180 when 295 tokens
[05/15 11:07:08 INFO agent] 5a8a9bc15542996c9b8d5f36: miss SP1 529355_0 (A Chorus Line) for 0.0063 <= 0.6348 when 295 tokens
[05/15 11:07:09 INFO agent] 5ae75a5a5542991bbc9761f3: remove SP1 38871163_0 (Kriti Sanon) from memory for 0.7051 -> 0.1247 <= 0.6918 when 333 tokens
[05/15 11:07:14 INFO agent] 5adfa451554299025d62a319: miss SP1 39793552_0 (Alleyne v. United States) for 0.0060 <= 0.5705 when 391 tokens
[05/15 11:07:14 INFO agent]


9: 748 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 59.20 | 71.07 | 73.49 |  72.27 |
|      Norm answer | 58.68 | 70.51 | 72.99 |  71.65 |
| Support sentence | 52.59 | 76.29 | 78.73 |  75.77 |
| Support sentence | 50.90 | 75.82 | 79.54 |  74.77 |
|  Support passage | 81.11 | 84.83 | 84.83 |  84.83 |
|            Joint | 38.34 | 62.91 | 66.99 |  63.48 |
3.135989196488859


[05/15 11:07:49 INFO agent] 5a7d19d85542995ed0d165e8: augment SP ['35001974_0'] into memory
[05/15 11:07:49 WARNING agent] 5a7d19d85542995ed0d165e8: augment false SP 35001974_0 into memory {'79371_0': 0.8471336364746094, '35001974_0': 0.29022541642189026}
[05/15 11:07:53 INFO agent] 5a7363ec5542991f29ee2dd7: miss SP1 109495_0 (Key West, Florida) for 0.0404 <= 0.5634 when 381 tokens
[05/15 11:07:55 INFO agent] 5ae75a5a5542991bbc9761f3: miss SP1 35121149_0 (Raabta (song)) for 0.2161 <= 0.6567 when 366 tokens
[05/15 11:08:01 INFO agent] 5ae5e762554299546bf82faf: miss SP1 2110323_0 (Rihanna) for 0.4111 <= 0.6750 when 309 tokens
[05/15 11:08:01 INFO agent] 5ae5e762554299546bf82faf: augment SP ['2110323_0'] into memory
[05/15 11:08:03 INFO agent] 5ac17b485542994ab5c67d65: miss SP0 23786794_0 (St Wilfrid's Church, Alford) for 0.0555 <= 0.6298 when 302 tokens
[05/15 11:08:10 INFO agent] 5a80b9ae5542992bc0c4a7eb: augment SP ['14900848_0'] into memory
[05/15 11:08:10 WARNING agent] 5a80b9ae55429


10: 721 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 59.43 | 71.34 | 73.77 |  72.54 |
|      Norm answer | 58.87 | 70.75 | 73.24 |  71.89 |
| Support sentence | 52.69 | 76.55 | 79.01 |  76.03 |
| Support sentence | 50.99 | 76.08 | 79.81 |  75.03 |
|  Support passage | 81.36 | 85.13 | 85.13 |  85.13 |
|            Joint | 38.43 | 63.13 | 67.24 |  63.70 |
3.236866981769075
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 60.90 | 73.44 | 75.94 |  74.74 |
|      Norm answer | 60.34 | 72.87 | 75.45 |  74.09 |
| Support sentence | 53.10 | 79.06 | 81.74 |  78.43 |
| Support sentence | 51.21 | 78.80 | 83.72 |  77.25 |
|  Support passage | 82.16 | 88.05 | 88.05 |  88.05 |
|            Joint | 38.65 | 64.17 | 68.38 |  64.74 |


In [45]:
MAX_OBS = 20
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

while len(observations) > 0 and n_observed < MAX_OBS:
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in tqdm(zip(q_ids, questions), desc='exe', total=len(q_ids)):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.execute(cmd, q_id, excluded=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    print(f"{n_observed}: {len(observations)} remained")
    if n_observed % 5 == 0 and split != 'test':
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()

[05/15 11:09:35 INFO agent] 5ab5b6c1554299637185c5e5: miss SP1 3139987_0 (Hellogoodbye) for 0.0624 <= 0.6184 when 500 tokens
[05/15 11:09:42 INFO agent] 5a79131a554299148911f9e3: augment SP ['1180262_0'] into memory
[05/15 11:09:42 WARNING agent] 5a79131a554299148911f9e3: augment false SP 1180262_0 into memory {'137840_0': 0.9642654657363892, '1180262_0': 0.20598077774047852}
[05/15 11:09:49 INFO agent] 5a750d315542996c70cfae84: augment SP ['274959_0'] into memory
[05/15 11:09:49 WARNING agent] 5a750d315542996c70cfae84: augment false SP 274959_0 into memory {'17056414_0': 0.9572461247444153, '274959_0': 1.0}
[05/15 11:09:49 INFO agent] 5ae6537c5542991bbc9760c1: remove SP1 525314_0 (Shrek 2) from memory for 0.7457 -> 0.0913 <= 0.6260 when 512 tokens
[05/15 11:09:54 INFO agent] 5a730abe5542994cef4bc42f: remove SP0 3800972_0 (Cricket fighting) from memory for 0.9141 -> 0.3620 <= 0.5009 when 273 tokens
[05/15 11:09:57 INFO agent] 5a82028a554299676cceb1d6: miss SP1 39364536_0 (The Curse of 


11: 704 remained


[05/15 11:10:14 INFO agent] 5ab7f0015542992aa3b8c88b: augment SP ['8597528_0'] into memory
[05/15 11:10:14 WARNING agent] 5ab7f0015542992aa3b8c88b: augment false SP 8597528_0 into memory {'6154275_0': 0.9288910627365112, '8597528_0': 0.06041277647018433}
[05/15 11:10:16 INFO agent] 5ac1a3665542994ab5c67daf: miss SP1 125267_0 (Prince Andrew, Duke of York) for 0.3717 <= 0.6024 when 428 tokens
[05/15 11:10:16 INFO agent] 5ac1a3665542994ab5c67daf: augment SP ['50564867_0'] into memory
[05/15 11:10:16 WARNING agent] 5ac1a3665542994ab5c67daf: augment false SP 50564867_0 into memory {'3652599_0': 0.9356913566589355, '50564867_0': 0.24793200492858888}
[05/15 11:10:20 INFO agent] 5ae75a5a5542991bbc9761f3: remove SP1 38871163_0 (Kriti Sanon) from memory for 0.7051 -> 0.0233 <= 0.6553 when 362 tokens
[05/15 11:10:20 INFO agent] 5abbc96755429931dba1452c: miss SP1 2627820_0 (Chris Brown) for 0.0113 <= 0.5993 when 362 tokens
[05/15 11:10:20 INFO agent] 5abbc96755429931dba1452c: augment SP ['2627820_


12: 681 remained


[05/15 11:11:01 INFO agent] 5ab42b24554299753aec5a43: miss SP1 33397730_0 (Operation Lighthouse (1937)) for 0.0955 <= 0.4777 when 346 tokens
[05/15 11:11:04 INFO agent] 5aba0c2655429955dce3edb7: miss SP1 43117009_0 (2015 Masters Tournament) for 0.1247 <= 0.6070 when 399 tokens
[05/15 11:11:04 INFO agent] 5a81d81e554299676cceb0f6: augment SP ['42254_0'] into memory
[05/15 11:11:04 WARNING agent] 5a81d81e554299676cceb0f6: augment false SP 42254_0 into memory {'39848_0': 0.9895926713943481, '42254_0': 0.0699278712272644}
[05/15 11:11:12 INFO agent] 5a7615fb5542992db9473727: remove SP1 37914_0 (Carmen) from memory for 0.9293 -> 0.4208 <= 0.5375 when 473 tokens
[05/15 11:11:17 INFO agent] 5ae5e8dc554299546bf82fbe: miss SP0 410591_0 (Maulana Karenga) for 0.0074 <= 0.7236 when 299 tokens
[05/15 11:11:17 INFO agent] 5a8760e75542996e4f3087af: miss SP0 442343_0 (Whatever Happened to... Robot Jones?) for 0.2115 <= 0.3878 when 299 tokens
[05/15 11:11:19 INFO agent] 5a843de75542992ef85e23b4: remove

...


18: 595 remained


[05/15 11:14:46 INFO agent] 5ae305df5542992decbdcdc3: miss SP1 34639380_0 (2012 Louisville Cardinals football team) for 0.1124 <= 0.6516 when 480 tokens
[05/15 11:14:48 INFO agent] 5a8dd5f955429917b4a5bcc7: miss SP0 34612184_0 (1998 Toyota Grand Prix of Long Beach) for 0.1974 <= 0.3660 when 379 tokens
[05/15 11:14:49 INFO agent] 5a7363ec5542991f29ee2dd7: augment SP ['841760_0'] into memory
[05/15 11:14:49 WARNING agent] 5a7363ec5542991f29ee2dd7: augment false SP 841760_0 into memory {'1500871_0': 0.9977765679359436, '841760_0': 0.3971134185791016}
[05/15 11:14:49 INFO agent] 5ae5d7b755429929b08079cd: augment SP ['519408_0'] into memory
[05/15 11:14:49 WARNING agent] 5ae5d7b755429929b08079cd: augment false SP 519408_0 into memory {'268705_0': 0.9895420074462891, '519408_0': 0.30695614814758304}
[05/15 11:14:58 INFO agent] 5a75140b5542996c70cfae87: remove SP1 8237_0 (Don Quixote) from memory for 0.7026 -> 0.0934 <= 0.5265 when 320 tokens
[05/15 11:15:02 INFO agent] 5ac4f12d55429924173fb4


19: 587 remained


[05/15 11:15:21 INFO agent] 5ae305df5542992decbdcdc3: miss SP1 35663654_0 (2013 Sugar Bowl) for 0.1181 <= 0.4922 when 450 tokens
[05/15 11:15:21 INFO agent] 5ade51b45542997c77adedf4: miss SP0 2258917_0 (CommunityAmerica Ballpark) for 0.0750 <= 0.5696 when 450 tokens
[05/15 11:15:24 INFO agent] 5ac1bff05542994ab5c67dfc: miss SP1 6523384_0 (Francisco Rafael Arellano Félix) for 0.0763 <= 0.5913 when 297 tokens
[05/15 11:15:24 INFO agent] 5ae2c696554299495565db32: remove SP0 3687575_0 (Thirlestane Castle) from memory for 0.7218 -> 0.4939 <= 0.5358 when 297 tokens
[05/15 11:15:30 INFO agent] 5a8dcbc0554299068b959df4: remove SP0 1212038_0 (Operation Gladio) from memory for 0.8560 -> 0.3241 <= 0.5313 when 331 tokens
[05/15 11:15:37 INFO agent] 5a76c98855429972597f13d0: augment SP ['86026_0'] into memory
[05/15 11:15:37 WARNING agent] 5a76c98855429972597f13d0: augment false SP 86026_0 into memory {'25427681_0': 0.44827914237976074, '17934171_0': 0.8875547051429749, '86026_0': 1.0}
[05/15 11:15


20: 574 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 60.41 | 72.55 | 75.05 |  73.78 |
|      Norm answer | 59.68 | 71.81 | 74.35 |  72.97 |
| Support sentence | 53.46 | 77.84 | 80.36 |  77.30 |
| Support sentence | 51.63 | 77.33 | 81.14 |  76.28 |
|  Support passage | 82.54 | 86.55 | 86.55 |  86.55 |
|            Joint | 38.96 | 64.09 | 68.26 |  64.67 |
4.112356515867657
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 61.84 | 74.47 | 76.99 |  75.77 |
|      Norm answer | 61.05 | 73.70 | 76.29 |  74.92 |
| Support sentence | 53.87 | 80.08 | 82.78 |  79.45 |
| Support sentence | 51.84 | 79.76 | 84.64 |  78.27 |
|  Support passage | 83.34 | 89.14 | 89.14 |  89.14 |
|            Joint | 39.18 | 65.09 | 69.34 |  65.67 |


In [46]:
MAX_OBS = 30
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

while len(observations) > 0 and n_observed < MAX_OBS:
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in tqdm(zip(q_ids, questions), desc='exe', total=len(q_ids)):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.execute(cmd, q_id, excluded=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    print(f"{n_observed}: {len(observations)} remained")
    if n_observed % 5 == 0  and split != 'test':
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()

[05/15 11:16:47 INFO agent] 5a8dd5f955429917b4a5bcc7: remove SP0 348734_0 (Alex Zanardi) from memory for 0.7860 -> 0.4383 <= 0.5782 when 317 tokens
[05/15 11:16:54 INFO agent] 5ab56f4b554299637185c599: miss SP0 37644013_0 (Republican Party presidential primaries, 2016) for 0.0068 <= 0.6057 when 348 tokens
[05/15 11:16:54 INFO agent] 5ab29ec6554299194fa93473: miss SP0 10304105_0 (Machete (2010 film)) for 0.1041 <= 0.4812 when 348 tokens
[05/15 11:16:58 INFO agent] 5a793d335542994bb9456fca: miss SP0 26146987_0 (The Sun (United Kingdom)) for 0.0500 <= 0.6225 when 351 tokens
[05/15 11:17:07 INFO agent] 5adf400a5542993a75d2644e: miss SP0 6906713_0 (German submarine U-32 (S182)) for 0.1919 <= 0.4442 when 286 tokens
[05/15 11:17:09 INFO agent] 5a7195a05542994082a3e881: augment SP ['30344963_0'] into memory
[05/15 11:17:09 WARNING agent] 5a7195a05542994082a3e881: augment false SP 30344963_0 into memory {'673848_0': 0.9898640513420105, '30344963_0': 1.0}
[05/15 11:17:14 INFO agent] 5ac19ff35542


21: 566 remained


[05/15 11:17:20 INFO agent] 5ab5b6c1554299637185c5e5: miss SP1 3139987_0 (Hellogoodbye) for 0.0624 <= 0.6184 when 310 tokens
[05/15 11:17:21 INFO agent] 5ac5275755429924173fb617: miss SP0 46340990_0 (Marc Schiller) for 0.0165 <= 0.7458 when 331 tokens
[05/15 11:17:32 INFO agent] 5a78c235554299078472775d: miss SP0 7999675_0 (Juan Mata) for 0.0262 <= 0.5081 when 512 tokens
[05/15 11:17:34 INFO agent] 5ae0ee2c5542993d6555eca3: remove SP0 466636_0 (Travis McGee) from memory for 0.7413 -> 0.0478 <= 0.6941 when 371 tokens
[05/15 11:17:37 INFO agent] 5a8b3de95542997f31a41cc3: miss SP0 455520_0 (Tian Tan Buddha) for 0.0701 <= 0.5586 when 288 tokens
[05/15 11:17:46 INFO agent] 5ae16d9655429920d52343ce: miss SP1 309962_0 (Coast to Coast AM) for 0.0215 <= 0.6930 when 453 tokens
[05/15 11:17:46 INFO agent] 5ae16d9655429920d52343ce: augment SP ['309962_0'] into memory
[05/15 11:17:46 INFO agent] 5ae0036a55429942ec259bdf: miss SP0 436948_0 (Argentine air forces in the Falklands War) for 0.0092 <= 0.


22: 557 remained


[05/15 11:17:51 INFO agent] 5a77724455429972597f153e: miss SP0 227965_0 (Indianapolis Motor Speedway) for 0.0444 <= 0.5138 when 420 tokens
[05/15 11:17:53 INFO agent] 5a77cb5e55429967ab1052a7: remove SP0 61166_0 (Dan Castellaneta) from memory for 0.9917 -> 0.2645 <= 0.6018 when 355 tokens
[05/15 11:17:58 INFO agent] 5abd85375542993062266cba: miss SP1 487318_0 (League of the Three Emperors) for 0.0043 <= 0.6346 when 466 tokens
[05/15 11:18:12 INFO agent] 5ab43b9d5542990594ba9bc3: augment SP ['66299_0'] into memory
[05/15 11:18:12 WARNING agent] 5ab43b9d5542990594ba9bc3: augment false SP 66299_0 into memory {'22897475_0': 0.9166759848594666, '66299_0': 0.1478052616119385}
[05/15 11:18:14 INFO agent] 5abd14c455429924427fceec: miss SP0 284357_0 (1812 Overture) for 0.0129 <= 0.6714 when 327 tokens
[05/15 11:18:18 INFO agent] 5ae10572554299422ee995cd: miss SP0 766372_0 (1966 Atlantic hurricane season) for 0.1285 <= 0.4542 when 369 tokens


...


28: 522 remained


[05/15 11:20:59 INFO agent] 5a8a9bc15542996c9b8d5f36: miss SP1 529355_0 (A Chorus Line) for 0.0063 <= 0.6348 when 294 tokens
[05/15 11:21:01 INFO agent] 5a7da3a95542990b8f5039e6: miss SP1 37537819_0 (Jim Kelch) for 0.1689 <= 0.3735 when 278 tokens
[05/15 11:21:05 INFO agent] 5a792ad055429907847277d1: remove SP0 30873367_0 (Mickey's PhilharMagic) from memory for 0.8246 -> 0.1715 <= 0.5375 when 324 tokens
[05/15 11:21:10 INFO agent] 5a777fd655429949eeb29e93: miss SP1 49188923_0 (Felix Avenue) for 0.1875 <= 0.6106 when 512 tokens
[05/15 11:21:10 INFO agent] 5a777fd655429949eeb29e93: augment SP ['49188923_0'] into memory
[05/15 11:21:13 INFO agent] 5a83d0ed5542996488c2e4e7: miss SP1 146918_0 (Homebrewing) for 0.0444 <= 0.5728 when 324 tokens
[05/15 11:21:13 INFO agent] 5a83d0ed5542996488c2e4e7: augment SP ['146918_0'] into memory



29: 513 remained


[05/15 11:21:25 INFO agent] 5ab31864554299233954ff06: miss SP1 17438452_0 (Apratim Majumdar) for 0.5462 <= 0.5699 when 325 tokens
[05/15 11:21:28 INFO agent] 5ae75a5a5542991bbc9761f3: remove SP1 38871163_0 (Kriti Sanon) from memory for 0.7051 -> 0.0381 <= 0.6416 when 349 tokens
[05/15 11:21:36 INFO agent] 5a78c235554299078472775d: miss SP0 45023134_0 (5 Magazine) for 0.2332 <= 0.4282 when 333 tokens



30: 508 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 60.96 | 73.20 | 75.72 |  74.43 |
|      Norm answer | 60.18 | 72.40 | 74.97 |  73.55 |
| Support sentence | 53.80 | 78.50 | 81.05 |  77.96 |
| Support sentence | 51.92 | 77.96 | 81.84 |  76.92 |
|  Support passage | 83.17 | 87.29 | 87.29 |  87.29 |
|            Joint | 39.19 | 64.62 | 68.83 |  65.20 |
4.844969615124915
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.32 | 75.04 | 77.59 |  76.36 |
|      Norm answer | 61.46 | 74.20 | 76.81 |  75.41 |
| Support sentence | 54.17 | 80.56 | 83.28 |  79.94 |
| Support sentence | 52.11 | 80.19 | 85.03 |  78.74 |
|  Support passage | 83.92 | 89.67 | 89.67 |  89.67 |
|            Joint | 39.38 | 65.56 | 69.85 |  66.15 |


In [47]:
MAX_OBS = 50
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

while len(observations) > 0 and n_observed < MAX_OBS:
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in tqdm(zip(q_ids, questions), desc='exe', total=len(q_ids)):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.execute(cmd, q_id, excluded=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    print(f"{n_observed}: {len(observations)} remained")
    if n_observed % 10 == 0  and split != 'test':
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()

[05/15 11:22:40 INFO agent] 5ae66b055542995703ce8b61: miss SP1 37439115_0 (Ray Cameron (comedian)) for 0.2946 <= 0.6793 when 286 tokens
[05/15 11:22:42 INFO agent] 5a7f85505542995d8a8dde95: remove SP0 1117114_0 (Point of Entry) from memory for 0.8905 -> 0.4855 <= 0.5218 when 309 tokens
[05/15 11:22:42 INFO agent] 5a8d1f015542994ba4e3dc08: remove SP0 197745_0 (Pyrrhus of Epirus) from memory for 0.5683 -> 0.5360 <= 0.5372 when 309 tokens
[05/15 11:22:43 INFO agent] 5a7cb94f554299683c1c6353: miss SP0 314508_0 (Afroman) for 0.3288 <= 0.4148 when 334 tokens
[05/15 11:22:48 INFO agent] 5a793d335542994bb9456fca: miss SP0 26146987_0 (The Sun (United Kingdom)) for 0.0500 <= 0.6225 when 341 tokens
[05/15 11:22:52 INFO agent] 5a8760e75542996e4f3087af: miss SP0 442343_0 (Whatever Happened to... Robot Jones?) for 0.2115 <= 0.3878 when 296 tokens
[05/15 11:22:55 INFO agent] 5abd882f55429924427fd045: miss SP0 11261237_0 (Birstall Shopping Park) for 0.0267 <= 0.5176 when 390 tokens
[05/15 11:22:57 INF


31: 504 remained


[05/15 11:23:08 INFO agent] 5a8f69b55542997ba9cb324f: augment SP ['28150413_0', '24971102_0'] into memory
[05/15 11:23:08 WARNING agent] 5a8f69b55542997ba9cb324f: augment false SP 24971102_0 into memory {'127032_0': 0.9870027899742126, '28150413_0': 0.31560462713241577, '24971102_0': 0.32286660671234135}
[05/15 11:23:11 INFO agent] 5a7f85505542995d8a8dde95: miss SP0 11940266_0 (Heading Out to the Highway) for 0.0393 <= 0.4447 when 318 tokens
[05/15 11:23:11 INFO agent] 5a7da3a95542990b8f5039e6: miss SP1 37537819_0 (Jim Kelch) for 0.1579 <= 0.4557 when 318 tokens
[05/15 11:23:15 INFO agent] 5ab29ec6554299194fa93473: miss SP0 10304105_0 (Machete (2010 film)) for 0.1041 <= 0.4812 when 403 tokens



32: 502 remained


[05/15 11:23:45 INFO agent] 5a88db445542993b751ca88a: miss SP0 48233724_0 (Picciola serenata) for 0.0252 <= 0.4764 when 348 tokens
[05/15 11:23:47 INFO agent] 5a8b86645542997f31a41d62: miss SP1 46299779_0 (Legends of Tomorrow) for 0.1272 <= 0.6497 when 512 tokens
[05/15 11:23:58 INFO agent] 5ab29842554299194fa93420: miss SP1 6780616_0 (La mamma morta) for 0.4173 <= 0.5869 when 284 tokens


...

[05/15 11:30:45 INFO agent] 5adf86b45542993344016cc1: augment SP ['149448_0'] into memory
[05/15 11:30:45 WARNING agent] 5adf86b45542993344016cc1: augment false SP 149448_0 into memory {'54386_0': 0.9820393919944763, '149448_0': 1.0}
[05/15 11:30:54 INFO agent] 5a8b6efd5542997f31a41d2b: augment SP ['16001406_0'] into memory
[05/15 11:30:54 WARNING agent] 5a8b6efd5542997f31a41d2b: augment false SP 16001406_0 into memory {'478401_0': 0.5705876350402832, '16001406_0': 1.0}
[05/15 11:30:54 INFO agent] 5adf400a5542993a75d2644e: miss SP1 6906713_0 (German submarine U-32 (S182)) for 0.1254 <= 0.4892 when 347 tokens
[05/15 11:30:57 INFO agent] 5a7cdd1c554299683c1c63a8: miss SP0 39146149_0 (The Smurfs: The Legend of Smurfy Hollow) for 0.0364 <= 0.5517 when 356 tokens



48: 440 remained



49: 434 remained


[05/15 11:31:41 INFO agent] 5ab3b42c5542992ade7c6e4f: remove SP0 73412_0 (North by Northwest) from memory for 0.8105 -> 0.4397 <= 0.5567 when 420 tokens



50: 432 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 61.43 | 73.80 | 76.32 |  75.06 |
|      Norm answer | 60.62 | 72.98 | 75.56 |  74.16 |
| Support sentence | 54.09 | 79.13 | 81.73 |  78.58 |
| Support sentence | 52.10 | 78.55 | 82.49 |  77.52 |
|  Support passage | 83.74 | 88.03 | 88.03 |  88.03 |
|            Joint | 39.42 | 65.08 | 69.31 |  65.67 |
6.114652261985145
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.62 | 75.35 | 77.90 |  76.66 |
|      Norm answer | 61.76 | 74.50 | 77.12 |  75.73 |
| Support sentence | 54.45 | 80.94 | 83.69 |  80.31 |
| Support sentence | 52.29 | 80.50 | 85.33 |  79.11 |
|  Support passage | 84.38 | 90.11 | 90.11 |  90.11 |
|            Joint | 39.61 | 65.88 | 70.20 |  66.47 |


In [48]:
MAX_OBS = 100
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

while len(observations) > 0 and n_observed < MAX_OBS:
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in tqdm(zip(q_ids, questions), desc='exe', total=len(q_ids)):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.execute(cmd, q_id, excluded=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    print(f"{n_observed}: {len(observations)} remained")
    if n_observed % 10 == 0 and split != 'test':
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()

[05/15 11:32:39 INFO agent] 5a7344e95542991f9a20c6ce: miss SP1 5637174_0 (SexyBack) for 0.1611 <= 0.5309 when 414 tokens



51: 431 remained


[05/15 11:33:01 INFO agent] 5a8f18d555429918e830d189: miss SP0 888461_0 (Caistor) for 0.4040 <= 0.4292 when 305 tokens
[05/15 11:33:06 INFO agent] 5a88db445542993b751ca88a: miss SP0 48233724_0 (Picciola serenata) for 0.0252 <= 0.4764 when 299 tokens



52: 428 remained


[05/15 11:33:28 INFO agent] 5a86681c5542991e77181644: augment SP ['15700250_0'] into memory
[05/15 11:33:28 WARNING agent] 5a86681c5542991e77181644: augment false SP 15700250_0 into memory {'29964347_0': 0.6868196725845337, '15700250_0': 1.0}
[05/15 11:33:32 INFO agent] 5a78c235554299078472775d: miss SP0 45023134_0 (5 Magazine) for 0.2332 <= 0.4282 when 512 tokens
[05/15 11:33:37 INFO agent] 5ade7feb5542992fa25da799: miss SP0 193362_0 (Minami-Tori-shima) for 0.3074 <= 0.3696 when 327 tokens
[05/15 11:33:40 INFO agent] 5a776f8555429972597f1534: remove SP0 124891_0 (Polson, Montana) from memory for 0.5882 -> 0.3346 <= 0.4945 when 333 tokens


...


98: 353 remained


[05/15 11:50:58 INFO agent] 5a77857155429949eeb29ebf: miss SP1 21844820_0 (U2 360° Tour) for 0.0122 <= 0.6818 when 315 tokens
[05/15 11:51:04 INFO agent] 5ac19ff35542991316484b61: remove SP0 6018749_0 (Phineas and Ferb) from memory for 0.7304 -> 0.4020 <= 0.4448 when 414 tokens



99: 353 remained


[05/15 11:51:08 INFO agent] 5a85ea095542994775f606a8: miss SP1 12342237_0 (The Hork-Bajir Chronicles) for 0.5066 <= 0.6124 when 512 tokens
[05/15 11:51:08 INFO agent] 5a85ea095542994775f606a8: augment SP ['12342237_0'] into memory
[05/15 11:51:08 INFO agent] 5a8dd5f955429917b4a5bcc7: miss SP1 34612184_0 (1998 Toyota Grand Prix of Long Beach) for 0.1815 <= 0.6143 when 512 tokens
[05/15 11:51:08 INFO agent] 5a8dd5f955429917b4a5bcc7: augment SP ['34612184_0'] into memory
[05/15 11:51:08 INFO agent] 5ab5b6c1554299637185c5e5: miss SP0 165681_0 (The Four Seasons (band)) for 0.0377 <= 0.5119 when 512 tokens
[05/15 11:51:08 INFO agent] 5a8b78775542997f31a41d3d: miss SP1 5004696_0 (Chimaji Appa) for 0.0135 <= 0.7216 when 512 tokens
[05/15 11:51:10 INFO agent] 5a7af4cd55429931da12c9a6: miss SP1 909937_0 (Daily Jang) for 0.0030 <= 0.6609 when 512 tokens
[05/15 11:51:10 INFO agent] 5a7af4cd55429931da12c9a6: augment SP ['909937_0'] into memory
[05/15 11:51:10 INFO agent] 5ae0a6ea5542993d6555ebe9: m


100: 339 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.00 | 74.48 | 77.02 |  75.76 |
|      Norm answer | 61.11 | 73.57 | 76.18 |  74.75 |
| Support sentence | 54.44 | 79.84 | 82.49 |  79.27 |
| Support sentence | 52.41 | 79.24 | 83.27 |  78.18 |
|  Support passage | 84.38 | 88.88 | 88.88 |  88.88 |
|            Joint | 39.70 | 65.60 | 69.88 |  66.18 |
8.74247130317353
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.89 | 75.71 | 78.27 |  77.01 |
|      Norm answer | 62.04 | 74.84 | 77.47 |  76.05 |
| Support sentence | 54.72 | 81.20 | 83.97 |  80.58 |
| Support sentence | 52.53 | 80.69 | 85.37 |  79.38 |
|  Support passage | 84.88 | 90.48 | 90.48 |  90.48 |
|            Joint | 39.84 | 66.21 | 70.54 |  66.79 |


In [49]:
MAX_OBS = 300
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

pbar = tqdm(total=MAX_OBS - n_observed)
while len(observations) > 0 and n_observed < MAX_OBS:
    pbar.set_description(f"act - {len(observations)}")
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0, disable_tqdm=True)
    pbar.set_description(f"exe - {len(observations)}")
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in zip(q_ids, questions):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.execute(cmd, q_id, excluded=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    pbar.update(1)
    if n_observed % 50 == 0 and split != 'test':
        print(f"{n_observed}: {len(observations)} remained")
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()
pbar.close()

[05/15 11:52:06 INFO agent] 5a75f79555429976ec32bcca: remove SP1 2027674_0 (1999 Odisha cyclone) from memory for 0.4560 -> 0.1676 <= 0.4227 when 473 tokens
[05/15 11:52:06 INFO agent] 5ade3d75554299728e26c68b: miss SP1 1165850_0 (Walt Disney Studios Park) for 0.2990 <= 0.6159 when 473 tokens
[05/15 11:52:09 INFO agent] 5a7da3a95542990b8f5039e6: miss SP1 37537819_0 (Jim Kelch) for 0.0894 <= 0.3758 when 399 tokens
[05/15 11:52:09 INFO agent] 5ab4147a5542996a3a969f1e: miss SP1 5608488_0 (Stephen Curry) for 0.1257 <= 0.4649 when 399 tokens
[05/15 11:52:15 INFO agent] 5a8a6b625542996c9b8d5ed9: miss SP1 3570185_0 (Pontins) for 0.2432 <= 0.6210 when 512 tokens
[05/15 11:52:15 INFO agent] 5a8a6b625542996c9b8d5ed9: augment SP ['3570185_0'] into memory
[05/15 11:52:19 INFO agent] 5a7bea3d5542996dd594b847: miss SP1 48161460_0 (Robin Hood (Once Upon a Time)) for 0.0799 <= 0.4685 when 408 tokens
[05/15 11:52:19 INFO agent] 5adea2435542992fa25da7e4: remove SP1 3220091_0 (Rio Airways) from memory for

150: 292 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.30 | 74.87 | 77.44 |  76.15 |
|      Norm answer | 61.40 | 73.93 | 76.57 |  75.10 |
| Support sentence | 54.67 | 80.25 | 82.92 |  79.69 |
| Support sentence | 52.53 | 79.62 | 83.63 |  78.60 |
|  Support passage | 84.77 | 89.35 | 89.35 |  89.35 |
|            Joint | 39.86 | 65.90 | 70.22 |  66.50 |
10.832815665091154


[05/15 12:07:19 INFO agent] 5a843de75542992ef85e23b4: miss SP1 697656_0 (Treasure Island Hotel and Casino) for 0.0507 <= 0.4698 when 314 tokens
[05/15 12:08:22 INFO agent] 5ae24b165542996483e64962: miss SP1 322104_0 (Friedrich Hund) for 0.4107 <= 0.5243 when 288 tokens
[05/15 12:08:22 INFO agent] 5ae24b165542996483e64962: augment SP ['322104_0'] into memory
[05/15 12:08:24 INFO agent] 5a7c5a87554299683c1c62eb: miss SP1 22851852_0 (Alamorio, California) for 0.0412 <= 0.5889 when 512 tokens
[05/15 12:08:39 INFO agent] 5ab298fb554299449642c922: remove SP0 16321_0 (Joseph McCarthy) from memory for 0.9328 -> 0.2111 <= 0.4782 when 289 tokens
[05/15 12:09:13 INFO agent] 5a78caa955429970f5fffd83: remove SP0 9455_0 (Etruscan language) from memory for 0.8801 -> 0.4993 <= 0.5305 when 449 tokens
[05/15 12:09:18 INFO agent] 5ae5e8dc554299546bf82fbe: miss SP1 410591_0 (Maulana Karenga) for 0.0937 <= 0.6196 when 315 tokens
[05/15 12:09:18 INFO agent] 5ae5e8dc554299546bf82fbe: augment SP ['20648200_0'

[05/15 12:20:40 INFO agent] 5ae0a59a55429945ae9593e2: augment SP ['13485_0'] into memory
[05/15 12:20:40 WARNING agent] 5ae0a59a55429945ae9593e2: augment false SP 13485_0 into memory {'455822_0': 0.9649125337600708, '13485_0': 0.4311870038509369}
[05/15 12:20:40 INFO agent] 5ab5b6c1554299637185c5e5: miss SP1 3139987_0 (Hellogoodbye) for 0.0375 <= 0.6258 when 512 tokens
[05/15 12:20:40 INFO agent] 5ade3d75554299728e26c68b: miss SP0 12797323_0 (Crush's Coaster) for 0.6005 <= 0.7229 when 512 tokens
[05/15 12:20:40 INFO agent] 5ae0a6ea5542993d6555ebe9: miss SP1 29578143_0 (Waste Land (film)) for 0.0213 <= 0.5819 when 512 tokens
[05/15 12:20:40 INFO agent] 5a901f735542990a98493591: miss SP1 32569_0 (Vitellius) for 0.0908 <= 0.5870 when 512 tokens
[05/15 12:20:42 INFO agent] 5ae73d235542991e8301cc29: miss SP1 43738974_0 (Kevin Loader) for 0.0595 <= 0.3888 when 512 tokens
[05/15 12:20:42 INFO agent] 5ae528ed5542993aec5ec16e: miss SP1 2613965_0 (Diana Quick) for 0.1953 <= 0.5768 when 512 token

200: 265 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.44 | 75.07 | 77.65 |  76.35 |
|      Norm answer | 61.49 | 74.07 | 76.71 |  75.25 |
| Support sentence | 54.75 | 80.45 | 83.12 |  79.89 |
| Support sentence | 52.59 | 79.81 | 83.85 |  78.78 |
|  Support passage | 84.90 | 89.57 | 89.57 |  89.57 |
|            Joint | 39.92 | 66.04 | 70.37 |  66.63 |
12.710195813639432


[05/15 12:21:02 INFO agent] 5ade3d75554299728e26c68b: remove SP0 1165850_0 (Walt Disney Studios Park) from memory for 0.7711 -> 0.0160 <= 0.6496 when 375 tokens
[05/15 12:21:05 INFO agent] 5abadc355542996606241644: miss SP1 22389647_0 (Florentius (consul 361)) for 0.3473 <= 0.5074 when 384 tokens
[05/15 12:21:05 INFO agent] 5abadc355542996606241644: remove SP0 25507_0 (Roman Empire) from memory for 0.7857 -> 0.3392 <= 0.5074 when 384 tokens
[05/15 12:21:09 INFO agent] 5ae142c455429920d5234355: miss SP2 2336320_0 (Rutgers University–Newark) for 0.1056 <= 0.4709 when 512 tokens
[05/15 12:21:12 INFO agent] 5ae406f055429970de88d874: remove SP1 1029471_0 (Roy Lassiter) from memory for 0.8333 -> 0.0405 <= 0.6686 when 378 tokens
[05/15 12:21:12 INFO agent] 5ae406f055429970de88d874: augment SP ['1029471_0'] into memory
[05/15 12:21:12 INFO agent] 5adfc06d55429906c02daa2a: remove SP0 1066561_0 (Boston Harbor) from memory for 0.9988 -> 0.0944 <= 0.5499 when 378 tokens
[05/15 12:21:15 INFO agent]

250: 255 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.50 | 75.15 | 77.72 |  76.43 |
|      Norm answer | 61.53 | 74.13 | 76.77 |  75.31 |
| Support sentence | 54.77 | 80.52 | 83.19 |  79.96 |
| Support sentence | 52.61 | 79.86 | 83.91 |  78.84 |
|  Support passage | 84.96 | 89.64 | 89.64 |  89.64 |
|            Joint | 39.93 | 66.09 | 70.42 |  66.69 |
14.466036461850102


[05/15 12:34:20 INFO agent] 5a7cb94f554299683c1c6353: miss SP0 314508_0 (Afroman) for 0.3288 <= 0.4148 when 418 tokens
[05/15 12:34:45 INFO agent] 5a79ecd05542996c55b2dca1: remove SP0 1577119_0 (Chris Weinke) from memory for 0.9902 -> 0.3200 <= 0.4948 when 501 tokens
[05/15 12:34:49 INFO agent] 5a8077ca5542992bc0c4a729: miss SP1 6683912_0 (Abraham Løkin) for 0.0012 <= 0.5362 when 315 tokens
[05/15 12:35:09 INFO agent] 5a7f38e255429930675136b6: miss SP1 9169947_0 (Ronkonkoma (LIRR station)) for 0.0540 <= 0.4256 when 512 tokens
[05/15 12:35:50 INFO agent] 5a84807d554299123d8c2278: miss SP0 54327840_0 (Oeuf (Hannibal)) for 0.1310 <= 0.3698 when 370 tokens
[05/15 12:35:57 INFO agent] 5abe3c7555429965af743ea5: miss SP1 10293_0 (Elbridge Gerry) for 0.0755 <= 0.5675 when 512 tokens
[05/15 12:36:44 INFO agent] 5ab3b42c5542992ade7c6e4f: remove SP0 73412_0 (North by Northwest) from memory for 0.8027 -> 0.4988 <= 0.5919 when 409 tokens
[05/15 12:36:59 INFO agent] 5ae142c455429920d5234355: miss SP

300: 249 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.55 | 75.21 | 77.79 |  76.50 |
|      Norm answer | 61.58 | 74.19 | 76.83 |  75.38 |
| Support sentence | 54.80 | 80.58 | 83.26 |  80.02 |
| Support sentence | 52.63 | 79.92 | 83.98 |  78.90 |
|  Support passage | 85.02 | 89.72 | 89.72 |  89.72 |
|            Joint | 39.96 | 66.15 | 70.48 |  66.74 |
16.17839297771776
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 63.23 | 76.12 | 78.71 |  77.44 |
|      Norm answer | 62.32 | 75.15 | 77.80 |  76.36 |
| Support sentence | 54.98 | 81.62 | 84.40 |  81.00 |
| Support sentence | 52.72 | 81.02 | 85.57 |  79.80 |
|  Support passage | 85.42 | 90.97 | 90.97 |  90.97 |
|            Joint | 40.03 | 66.59 | 70.98 |  67.18 |



In [50]:
MAX_OBS = 500
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

pbar = tqdm(total=MAX_OBS - n_observed)
while len(observations) > 0 and n_observed < MAX_OBS:
    pbar.set_description(f"act - {len(observations)}")
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0, disable_tqdm=True)
    pbar.set_description(f"exe - {len(observations)}")
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in zip(q_ids, questions):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.execute(cmd, q_id, excluded=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    pbar.update(1)
    if n_observed % 50 == 0 and split != 'test':
        print(f"{n_observed}: {len(observations)} remained")
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()
pbar.close()

[05/15 12:47:21 INFO agent] 5abadc355542996606241644: remove SP0 25507_0 (Roman Empire) from memory for 0.8461 -> 0.2257 <= 0.5097 when 343 tokens
[05/15 12:47:25 INFO agent] 5ab3b42c5542992ade7c6e4f: remove SP0 73412_0 (North by Northwest) from memory for 0.6103 -> 0.2829 <= 0.4896 when 412 tokens
[05/15 12:47:25 INFO agent] 5a82028a554299676cceb1d6: remove SP0 237274_0 (Thriller (genre)) from memory for 0.6078 -> 0.2851 <= 0.4483 when 412 tokens
[05/15 12:47:27 INFO agent] 5a7bea3d5542996dd594b847: miss SP1 3275778_0 (Tom Ellis (actor)) for 0.0115 <= 0.7671 when 344 tokens
[05/15 12:47:27 INFO agent] 5adea2435542992fa25da7e4: remove SP1 3220091_0 (Rio Airways) from memory for 0.8731 -> 0.0303 <= 0.6610 when 344 tokens
[05/15 12:47:30 INFO agent] 5adff34555429942ec259baa: remove SP0 31788031_0 (Bobby Noble (footballer, born 1949)) from memory for 0.8194 -> 0.0585 <= 0.5844 when 427 tokens
[05/15 12:47:30 INFO agent] 5a8e00515542995a26add454: miss SP1 33884405_0 (Derek Branning) for 0.

350: 243 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.62 | 75.29 | 77.86 |  76.58 |
|      Norm answer | 61.65 | 74.27 | 76.90 |  75.46 |
| Support sentence | 54.87 | 80.66 | 83.33 |  80.09 |
| Support sentence | 52.68 | 79.99 | 84.05 |  78.97 |
|  Support passage | 85.09 | 89.79 | 89.79 |  89.79 |
|            Joint | 40.01 | 66.21 | 70.54 |  66.81 |
17.83443619176232


[05/15 13:00:21 INFO agent] 5ac4e9ab5542996feb3fe974: remove SP1 1723963_0 (Roger Stone) from memory for 0.9605 -> 0.2018 <= 0.6525 when 423 tokens
[05/15 13:00:23 INFO agent] 5a7780e855429949eeb29e9f: miss SP0 32832973_0 (Anna Karenina (2012 film)) for 0.2033 <= 0.3235 when 366 tokens
[05/15 13:00:38 INFO agent] 5a7cdd1c554299683c1c63a8: miss SP0 39146149_0 (The Smurfs: The Legend of Smurfy Hollow) for 0.0364 <= 0.5517 when 325 tokens
[05/15 13:00:46 INFO agent] 5addf6135542990dbb2f7f23: miss SP1 49574937_0 (Discwoman) for 0.0055 <= 0.6056 when 389 tokens
[05/15 13:00:53 INFO agent] 5a78f05155429970f5fffde3: miss SP1 31460_0 (Tom Cruise) for 0.1065 <= 0.6299 when 355 tokens
[05/15 13:02:18 INFO agent] 5ade66f055429939a52fe8a0: remove SP0 1997230_0 (Humberside Airport) from memory for 0.9100 -> 0.3285 <= 0.5349 when 328 tokens
[05/15 13:02:24 INFO agent] 5a7b81715542997c3ec971d4: augment SP ['5968971_0'] into memory
[05/15 13:02:24 WARNING agent] 5a7b81715542997c3ec971d4: augment false

400: 233 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.66 | 75.35 | 77.92 |  76.65 |
|      Norm answer | 61.69 | 74.33 | 76.96 |  75.54 |
| Support sentence | 54.91 | 80.73 | 83.42 |  80.16 |
| Support sentence | 52.72 | 80.07 | 84.15 |  79.03 |
|  Support passage | 85.16 | 89.89 | 89.89 |  89.89 |
|            Joint | 40.03 | 66.24 | 70.57 |  66.85 |
19.458474004051318


[05/15 13:11:52 INFO agent] 5a8b452655429971feec468e: remove SP0 525755_0 (Kevin Garnett) from memory for 0.9365 -> 0.3190 <= 0.4736 when 501 tokens
[05/15 13:12:28 INFO agent] 5a82028a554299676cceb1d6: remove SP0 237274_0 (Thriller (genre)) from memory for 0.6409 -> 0.4424 <= 0.4892 when 359 tokens
[05/15 13:12:28 INFO agent] 5a843de75542992ef85e23b4: remove SP0 698467_0 (Steve Wynn) from memory for 0.9354 -> 0.4073 <= 0.4125 when 359 tokens
[05/15 13:12:44 INFO agent] 5a823fc055429940e5e1a810: remove SP0 3766171_0 (Norse–Gaels) from memory for 0.6129 -> 0.2935 <= 0.4549 when 318 tokens
[05/15 13:12:44 INFO agent] 5ab56200554299494045ef88: remove SP0 35022267_0 (Boules at the 1900 Summer Olympics) from memory for 0.7464 -> 0.1167 <= 0.5680 when 279 tokens
[05/15 13:13:25 INFO agent] 5adfc06d55429906c02daa2a: remove SP0 1066561_0 (Boston Harbor) from memory for 0.9394 -> 0.4070 <= 0.5822 when 376 tokens
[05/15 13:13:31 INFO agent] 5ab4147a5542996a3a969f1e: miss SP0 5608488_0 (Stephen C

450: 230 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.69 | 75.37 | 77.94 |  76.68 |
|      Norm answer | 61.70 | 74.34 | 76.97 |  75.55 |
| Support sentence | 54.92 | 80.76 | 83.45 |  80.19 |
| Support sentence | 52.73 | 80.09 | 84.18 |  79.06 |
|  Support passage | 85.19 | 89.93 | 89.93 |  89.93 |
|            Joint | 40.04 | 66.26 | 70.60 |  66.87 |
21.023362592842673


[05/15 13:24:53 INFO agent] 5a74e9c255429974ef308c93: miss SP1 36534077_0 (Mohsin Fani) for 0.1035 <= 0.4214 when 512 tokens
[05/15 13:25:11 INFO agent] 5ac325495542995ef918c12e: miss SP1 12458720_0 (Akrofuom (Ghana parliament constituency)) for 0.1640 <= 0.5160 when 454 tokens
[05/15 13:26:52 INFO agent] 5a901f735542990a98493591: remove SP0 32569_0 (Vitellius) from memory for 0.9283 -> 0.2857 <= 0.4620 when 374 tokens
[05/15 13:28:34 INFO agent] 5a74e9c255429974ef308c93: remove SP1 3888089_0 (Dabestan-e Mazaheb) from memory for 0.9664 -> 0.3026 <= 0.5767 when 512 tokens
[05/15 13:28:39 INFO agent] 5ab2d3c5554299295394687a: miss SP1 101393_0 (Liev Schreiber) for 0.0016 <= 0.5588 when 372 tokens
[05/15 13:28:53 INFO agent] 5ac3aa3655429939154138a2: miss SP0 30677885_0 (Ministry of European Integration (Albania)) for 0.2004 <= 0.4488 when 364 tokens
[05/15 13:29:02 INFO agent] 5a7e32905542991319bc943b: miss SP1 24608653_0 (2007 Trail Appliances Autumn Gold Curling Classic) for 0.0348 <= 

500: 228 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.71 | 75.40 | 77.97 |  76.71 |
|      Norm answer | 61.72 | 74.36 | 76.99 |  75.56 |
| Support sentence | 54.94 | 80.78 | 83.47 |  80.21 |
| Support sentence | 52.73 | 80.11 | 84.20 |  79.08 |
|  Support passage | 85.20 | 89.95 | 89.95 |  89.95 |
|            Joint | 40.05 | 66.28 | 70.62 |  66.89 |
22.56866981769075
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 63.32 | 76.26 | 78.85 |  77.59 |
|      Norm answer | 62.40 | 75.27 | 77.91 |  76.48 |
| Support sentence | 55.08 | 81.72 | 84.50 |  81.10 |
| Support sentence | 52.79 | 81.09 | 85.62 |  79.87 |
|  Support passage | 85.51 | 91.06 | 91.06 |  91.06 |
|            Joint | 40.12 | 66.71 | 71.09 |  67.31 |



In [51]:
MAX_OBS = 700
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

pbar = tqdm(total=MAX_OBS - n_observed)
while len(observations) > 0 and n_observed < MAX_OBS:
    pbar.set_description(f"act - {len(observations)}")
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0, disable_tqdm=True)
    pbar.set_description(f"exe - {len(observations)}")
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in zip(q_ids, questions):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.execute(cmd, q_id, excluded=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    pbar.update(1)
    if n_observed % 50 == 0 and split != 'test':
        print(f"{n_observed}: {len(observations)} remained")
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()
pbar.close()

[05/15 13:35:32 INFO agent] 5ae305df5542992decbdcdc3: miss SP1 34639380_0 (2012 Louisville Cardinals football team) for 0.4498 <= 0.6352 when 348 tokens
[05/15 13:35:34 INFO agent] 5a84807d554299123d8c2278: remove SP0 593041_0 (Hugh Dancy) from memory for 0.9202 -> 0.3004 <= 0.5744 when 404 tokens
[05/15 13:35:40 INFO agent] 5a82028a554299676cceb1d6: remove SP0 237274_0 (Thriller (genre)) from memory for 0.7675 -> 0.4660 <= 0.4951 when 349 tokens
[05/15 13:35:42 INFO agent] 5adfc06d55429906c02daa2a: remove SP0 1066561_0 (Boston Harbor) from memory for 0.9990 -> 0.5260 <= 0.5762 when 350 tokens
[05/15 13:35:44 INFO agent] 5ac3b8b8554299741d48a301: miss SP1 3032135_0 (Little Big League) for 0.0514 <= 0.2702 when 512 tokens
[05/15 13:35:53 INFO agent] 5a74e9c255429974ef308c93: remove SP1 3888089_0 (Dabestan-e Mazaheb) from memory for 0.9781 -> 0.4474 <= 0.5077 when 512 tokens
[05/15 13:35:57 INFO agent] 5adff34555429942ec259baa: remove SP0 31788031_0 (Bobby Noble (footballer, born 1949)) 

550: 223 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.73 | 75.43 | 78.01 |  76.73 |
|      Norm answer | 61.73 | 74.38 | 77.02 |  75.59 |
| Support sentence | 54.95 | 80.82 | 83.51 |  80.24 |
| Support sentence | 52.75 | 80.15 | 84.23 |  79.12 |
|  Support passage | 85.23 | 89.99 | 89.99 |  89.99 |
|            Joint | 40.05 | 66.31 | 70.65 |  66.91 |
24.091289669142473


[05/15 13:48:17 INFO agent] 5a7da3a95542990b8f5039e6: miss SP1 37537819_0 (Jim Kelch) for 0.0938 <= 0.4065 when 378 tokens
[05/15 13:48:23 INFO agent] 5a8284b555429954d2e2eb5f: remove SP0 705273_0 (Tropicana Las Vegas) from memory for 0.8961 -> 0.4779 <= 0.5120 when 361 tokens
[05/15 13:49:05 INFO agent] 5a7780e855429949eeb29e9f: remove SP0 11402229_0 (Aaron Taylor-Johnson) from memory for 0.5825 -> 0.3317 <= 0.4203 when 365 tokens
[05/15 13:49:51 INFO agent] 5a7da3a95542990b8f5039e6: miss SP1 37537819_0 (Jim Kelch) for 0.0947 <= 0.4998 when 400 tokens
[05/15 13:50:39 INFO agent] 5ac3a0db554299391541383a: miss SP0 254414_0 (Minnesota House of Representatives) for 0.0130 <= 0.6663 when 370 tokens
[05/15 13:50:39 INFO agent] 5ab56200554299494045ef88: remove SP1 35022267_0 (Boules at the 1900 Summer Olympics) from memory for 0.9667 -> 0.2998 <= 0.7090 when 370 tokens
[05/15 13:51:15 INFO agent] 5a74e9c255429974ef308c93: miss SP1 36534077_0 (Mohsin Fani) for 0.1035 <= 0.4214 when 512 token

600: 220 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.74 | 75.45 | 78.03 |  76.76 |
|      Norm answer | 61.74 | 74.41 | 77.04 |  75.61 |
| Support sentence | 54.95 | 80.84 | 83.54 |  80.26 |
| Support sentence | 52.75 | 80.17 | 84.26 |  79.13 |
|  Support passage | 85.24 | 90.01 | 90.01 |  90.01 |
|            Joint | 40.05 | 66.32 | 70.66 |  66.92 |
25.594598244429438


[05/15 13:58:16 INFO agent] 5ae305df5542992decbdcdc3: miss SP1 34639380_0 (2012 Louisville Cardinals football team) for 0.3524 <= 0.5941 when 386 tokens
[05/15 13:58:16 INFO agent] 5ae66b055542995703ce8b61: miss SP1 37439115_0 (Ray Cameron (comedian)) for 0.1700 <= 0.4991 when 386 tokens
[05/15 13:58:25 INFO agent] 5adf400a5542993a75d2644e: miss SP1 195728_0 (Type 212 submarine) for 0.3135 <= 0.5184 when 477 tokens
[05/15 13:58:27 INFO agent] 5adff34555429942ec259baa: remove SP1 31788031_0 (Bobby Noble (footballer, born 1949)) from memory for 0.6709 -> 0.1074 <= 0.4557 when 372 tokens
[05/15 13:58:40 INFO agent] 5adfc06d55429906c02daa2a: remove SP0 1066561_0 (Boston Harbor) from memory for 0.8366 -> 0.1762 <= 0.5340 when 339 tokens
[05/15 13:58:40 INFO agent] 5a823fc055429940e5e1a810: remove SP0 3766171_0 (Norse–Gaels) from memory for 0.5703 -> 0.3223 <= 0.4561 when 339 tokens
[05/15 13:58:51 INFO agent] 5a82028a554299676cceb1d6: remove SP0 237274_0 (Thriller (genre)) from memory for 0

650: 219 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.75 | 75.47 | 78.04 |  76.77 |
|      Norm answer | 61.76 | 74.42 | 77.05 |  75.63 |
| Support sentence | 54.95 | 80.85 | 83.55 |  80.28 |
| Support sentence | 52.75 | 80.18 | 84.27 |  79.15 |
|  Support passage | 85.25 | 90.03 | 90.03 |  90.03 |
|            Joint | 40.05 | 66.33 | 70.67 |  66.93 |
27.07981093855503


[05/15 14:10:11 INFO agent] 5ae305df5542992decbdcdc3: miss SP0 35663654_0 (2013 Sugar Bowl) for 0.1274 <= 0.4245 when 406 tokens
[05/15 14:11:26 INFO agent] 5a7cdd1c554299683c1c63a8: remove SP0 84791_0 (The Legend of Sleepy Hollow) from memory for 0.8855 -> 0.5390 <= 0.5787 when 345 tokens
[05/15 14:12:44 INFO agent] 5a8c7057554299653c1aa066: miss SP1 17401191_0 (2011 UEFA Champions League Final) for 0.0050 <= 0.6001 when 315 tokens
[05/15 14:14:41 INFO agent] 5a869e845542991e7718166f: miss SP1 33361400_0 (Ultra Lover) for 0.3797 <= 0.5665 when 342 tokens
[05/15 14:14:41 INFO agent] 5a869e845542991e7718166f: augment SP ['33361400_0'] into memory
[05/15 14:16:04 INFO agent] 5ae1b44c554299234fd042cd: miss SP1 39229756_0 (Adam McKenzie) for 0.0649 <= 0.5994 when 341 tokens
[05/15 14:16:18 INFO agent] 5ade66f055429939a52fe8a0: miss SP1 1370613_0 (Eastern Airways) for 0.0313 <= 0.6800 when 367 tokens
[05/15 14:17:50 INFO agent] 5a84d71c5542997b5ce3ff75: miss SP1 28953_0 (Stephen, King of En

700: 211 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.81 | 75.53 | 78.11 |  76.84 |
|      Norm answer | 61.81 | 74.47 | 77.11 |  75.68 |
| Support sentence | 54.98 | 80.92 | 83.62 |  80.34 |
| Support sentence | 52.75 | 80.24 | 84.34 |  79.21 |
|  Support passage | 85.31 | 90.11 | 90.11 |  90.11 |
|            Joint | 40.07 | 66.38 | 70.72 |  66.98 |
28.53869007427414
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 63.35 | 76.32 | 78.91 |  77.64 |
|      Norm answer | 62.40 | 75.30 | 77.95 |  76.52 |
| Support sentence | 55.12 | 81.77 | 84.55 |  81.15 |
| Support sentence | 52.82 | 81.14 | 85.62 |  79.95 |
|  Support passage | 85.60 | 91.12 | 91.12 |  91.12 |
|            Joint | 40.14 | 66.76 | 71.15 |  67.35 |



In [52]:
MAX_OBS = 1000
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

pbar = tqdm(total=MAX_OBS - n_observed)
while len(observations) > 0 and n_observed < MAX_OBS:
    pbar.set_description(f"act - {len(observations)}")
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0, disable_tqdm=True)
    pbar.set_description(f"exe - {len(observations)}")
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in zip(q_ids, questions):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.execute(cmd, q_id, excluded=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    pbar.update(1)
    if n_observed % 50 == 0 and split != 'test':
        print(f"{n_observed}: {len(observations)} remained")
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()
pbar.close()

[05/15 14:21:07 INFO agent] 5a7da3a95542990b8f5039e6: miss SP1 37537819_0 (Jim Kelch) for 0.1105 <= 0.3759 when 366 tokens
[05/15 14:21:28 INFO agent] 5adfc06d55429906c02daa2a: remove SP0 1066561_0 (Boston Harbor) from memory for 0.7842 -> 0.1483 <= 0.5730 when 418 tokens
[05/15 14:21:28 INFO agent] 5a823fc055429940e5e1a810: remove SP0 3766171_0 (Norse–Gaels) from memory for 0.5887 -> 0.3416 <= 0.4692 when 418 tokens
[05/15 14:21:30 INFO agent] 5ab56200554299494045ef88: remove SP0 35022267_0 (Boules at the 1900 Summer Olympics) from memory for 0.9230 -> 0.5162 <= 0.5370 when 387 tokens
[05/15 14:22:10 INFO agent] 5a7283355542991f9a20c4b1: augment SP ['1467835_0', '27485516_0'] into memory
[05/15 14:22:10 WARNING agent] 5a7283355542991f9a20c4b1: augment false SP 27485516_0 into memory {'41203018_0': 0.9545116424560547, '1467835_0': 0.32280802726745605, '27485516_0': 0.3383638143539429}
[05/15 14:22:53 INFO agent] 5abf8ae85542990832d3a14b: miss SP1 39348370_0 (Aleksei Gan) for 0.1155 <= 

750: 208 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.82 | 75.55 | 78.13 |  76.86 |
|      Norm answer | 61.81 | 74.47 | 77.11 |  75.68 |
| Support sentence | 54.98 | 80.94 | 83.65 |  80.36 |
| Support sentence | 52.75 | 80.26 | 84.36 |  79.22 |
|  Support passage | 85.32 | 90.14 | 90.14 |  90.14 |
|            Joint | 40.07 | 66.39 | 70.74 |  66.99 |
29.952734638757597


[05/15 14:32:08 INFO agent] 5ab29ec6554299194fa93473: miss SP0 10304105_0 (Machete (2010 film)) for 0.1041 <= 0.4812 when 274 tokens
[05/15 14:32:23 INFO agent] 5ab3b42c5542992ade7c6e4f: miss SP1 73412_0 (North by Northwest) for 0.3918 <= 0.6092 when 512 tokens
[05/15 14:32:58 INFO agent] 5abff0625542997ec76fd476: miss SP1 2805673_0 (Holly Ridge, Mississippi) for 0.1020 <= 0.6178 when 374 tokens
[05/15 14:33:10 INFO agent] 5a7cb94f554299683c1c6353: miss SP0 11615904_0 (The Hungry Hustlerz: Starvation Is Motivation) for 0.0124 <= 0.7659 when 368 tokens
[05/15 14:33:56 INFO agent] 5ac3b8b8554299741d48a301: miss SP1 3032135_0 (Little Big League) for 0.0500 <= 0.5209 when 345 tokens
[05/15 14:34:15 INFO agent] 5a7f54665542992097ad2f1a: remove SP1 15895358_0 (Calgary) from memory for 0.8585 -> 0.0733 <= 0.6684 when 281 tokens
[05/15 14:34:45 INFO agent] 5a8284b555429954d2e2eb5f: remove SP0 705273_0 (Tropicana Las Vegas) from memory for 0.9438 -> 0.5002 <= 0.5205 when 332 tokens
[05/15 14:38

800: 206 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.82 | 75.56 | 78.14 |  76.87 |
|      Norm answer | 61.81 | 74.48 | 77.12 |  75.69 |
| Support sentence | 54.98 | 80.96 | 83.67 |  80.37 |
| Support sentence | 52.75 | 80.27 | 84.38 |  79.24 |
|  Support passage | 85.32 | 90.15 | 90.15 |  90.15 |
|            Joint | 40.07 | 66.40 | 70.76 |  66.99 |
31.355975692099932


[05/15 14:42:49 INFO agent] 5ab6cd0e554299710c8d1f68: miss SP1 27977741_0 (Cristian Jeandet) for 0.0469 <= 0.5237 when 311 tokens
[05/15 14:42:49 INFO agent] 5adea2435542992fa25da7e4: remove SP1 3220091_0 (Rio Airways) from memory for 0.7461 -> 0.0330 <= 0.6610 when 311 tokens
[05/15 14:42:51 INFO agent] 5adfc06d55429906c02daa2a: remove SP0 1066561_0 (Boston Harbor) from memory for 0.9991 -> 0.2113 <= 0.5434 when 432 tokens
[05/15 14:42:51 INFO agent] 5a823fc055429940e5e1a810: remove SP0 3766171_0 (Norse–Gaels) from memory for 0.7302 -> 0.4554 <= 0.5448 when 432 tokens
[05/15 14:42:51 INFO agent] 5ac0b8395542996f0d89cc21: remove SP0 11884_0 (German language) from memory for 0.6995 -> 0.2956 <= 0.5235 when 432 tokens
[05/15 14:42:51 INFO agent] 5adff34555429942ec259baa: remove SP1 31788031_0 (Bobby Noble (footballer, born 1949)) from memory for 0.6111 -> 0.2277 <= 0.6111 when 432 tokens
[05/15 14:42:52 INFO agent] 5ab56200554299494045ef88: remove SP0 35022267_0 (Boules at the 1900 Summe

...

900: 194 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.90 | 75.67 | 78.24 |  76.97 |
|      Norm answer | 61.86 | 74.55 | 77.19 |  75.76 |
| Support sentence | 55.03 | 81.06 | 83.77 |  80.48 |
| Support sentence | 52.79 | 80.37 | 84.49 |  79.34 |
|  Support passage | 85.39 | 90.26 | 90.26 |  90.26 |
|            Joint | 40.12 | 66.48 | 70.84 |  67.08 |
34.056718433490886


[05/15 15:03:49 INFO agent] 5ab56200554299494045ef88: remove SP1 35022267_0 (Boules at the 1900 Summer Olympics) from memory for 0.8736 -> 0.4905 <= 0.5788 when 344 tokens
[05/15 15:03:57 INFO agent] 5abafb6d5542992ccd8e7e92: remove SP0 33303392_0 (Starship Entertainment) from memory for 0.9780 -> 0.0439 <= 0.5720 when 374 tokens
[05/15 15:04:01 INFO agent] 5a823fc055429940e5e1a810: remove SP0 3766171_0 (Norse–Gaels) from memory for 0.5445 -> 0.4349 <= 0.4673 when 316 tokens
[05/15 15:04:01 INFO agent] 5adff34555429942ec259baa: remove SP0 31788031_0 (Bobby Noble (footballer, born 1949)) from memory for 0.9202 -> 0.0084 <= 0.8417 when 316 tokens
[05/15 15:04:11 INFO agent] 5adfc06d55429906c02daa2a: remove SP0 1066561_0 (Boston Harbor) from memory for 0.9214 -> 0.3318 <= 0.5661 when 340 tokens
[05/15 15:04:22 INFO agent] 5a82028a554299676cceb1d6: remove SP0 237274_0 (Thriller (genre)) from memory for 0.6409 -> 0.4641 <= 0.5046 when 421 tokens
[05/15 15:04:25 INFO agent] 5ac3d1d9554299204

950: 191 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.94 | 75.71 | 78.28 |  77.01 |
|      Norm answer | 61.86 | 74.56 | 77.19 |  75.76 |
| Support sentence | 55.03 | 81.08 | 83.80 |  80.49 |
| Support sentence | 52.79 | 80.39 | 84.51 |  79.36 |
|  Support passage | 85.40 | 90.28 | 90.28 |  90.28 |
|            Joint | 40.12 | 66.50 | 70.86 |  67.09 |
35.35367994598244


[05/15 15:15:10 INFO agent] 5ae66b055542995703ce8b61: remove SP0 10637487_0 (Michael McIntyre) from memory for 0.8405 -> 0.4138 <= 0.4669 when 384 tokens
[05/15 15:15:15 INFO agent] 5abe3c7555429965af743ea5: miss SP1 10293_0 (Elbridge Gerry) for 0.1011 <= 0.6107 when 351 tokens
[05/15 15:15:29 INFO agent] 5a89d36e554299515336132a: miss SP0 2863155_0 (5.15) for 0.4399 <= 0.4415 when 338 tokens
[05/15 15:15:31 INFO agent] 5a84d71c5542997b5ce3ff75: miss SP1 15378755_0 (England in the High Middle Ages) for 0.1483 <= 0.5660 when 501 tokens
[05/15 15:18:37 INFO agent] 5ab8338155429919ba4e2260: miss SP0 4542403_0 (Mark Brayshaw) for 0.1216 <= 0.5094 when 343 tokens
[05/15 15:20:27 INFO agent] 5a739b1955429978a71e9044: remove SP0 198153_0 (Canning) from memory for 0.9800 -> 0.4133 <= 0.4564 when 379 tokens
[05/15 15:20:45 INFO agent] 5a7780e855429949eeb29e9f: miss SP1 11402229_0 (Aaron Taylor-Johnson) for 0.1281 <= 0.5134 when 369 tokens
[05/15 15:21:14 INFO agent] 5a739b1955429978a71e9044: mi

1000: 190 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 62.94 | 75.71 | 78.28 |  77.01 |
|      Norm answer | 61.86 | 74.56 | 77.19 |  75.76 |
| Support sentence | 55.03 | 81.08 | 83.80 |  80.49 |
| Support sentence | 52.79 | 80.39 | 84.51 |  79.36 |
|  Support passage | 85.40 | 90.28 | 90.28 |  90.28 |
|            Joint | 40.12 | 66.50 | 70.86 |  67.09 |
36.643349088453746
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 63.50 | 76.45 | 79.04 |  77.76 |
|      Norm answer | 62.43 | 75.34 | 77.99 |  76.55 |
| Support sentence | 55.14 | 81.85 | 84.64 |  81.23 |
| Support sentence | 52.84 | 81.17 | 85.62 |  80.00 |
|  Support passage | 85.69 | 91.21 | 91.21 |  91.21 |
|            Joint | 40.18 | 66.87 | 71.27 |  67.45 |



In [ ]:
assert False

In [ ]:
MAX_OBS = 1500
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

pbar = tqdm(total=MAX_OBS - n_observed)
while len(observations) > 0 and n_observed < MAX_OBS:
    pbar.set_description(f"act - {len(observations)}")
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    pbar.set_description(f"exe - {len(observations)}")
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in zip(q_ids, questions):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.execute(cmd, q_id, excluded=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    pbar.update(1)
    if n_observed % 100 == 0 and split != 'test':
        print(f"{n_observed}: {len(observations)} remained")
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()
pbar.close()

In [ ]:
MAX_OBS = 2000
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

pbar = tqdm(total=MAX_OBS - n_observed)
while len(observations) > 0 and n_observed < MAX_OBS:
    pbar.set_description(f"act - {len(observations)}")
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    pbar.set_description(f"exe - {len(observations)}")
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in zip(q_ids, questions):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.execute(cmd, q_id, excluded=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    pbar.update(1)
    if n_observed % 100 == 0 and split != 'test':
        print(f"{n_observed}: {len(observations)} remained")
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()
pbar.close()

# Analyze

In [202]:
q_ids = ['321321']
questions = ['What movie directed by Pitof in 2004 has a tie-in electronic game?']
observations = [None] * len(q_ids)

n_observed= 0

In [203]:
MAX_OBS = 100
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

while len(observations) > 0 and n_observed < MAX_OBS:
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in tqdm(zip(q_ids, questions), desc='exe', total=len(q_ids), disable=True):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.execute(cmd, q_id, excluded=agent.memory[q_id])  # xxx observed memory
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    print(f"{n_observed}: {len(observations)} remained")

1: 1 remained
2: 1 remained
3: 1 remained
4: 1 remained
5: 0 remained


## Error types

In [55]:
print({k: len(v) for k, v in agent.cases.items()})

{'miss_ans': 1019, 'add_dis': 1061, 'miss_sp': 449, 'false_expansion': 437, 'early_ans': 2, 'good_ans': 5051, 'gold_spp': 6312, 'bad_sp': 2237, 'aug_false_spp': 245, 'miss_sp2': 162, 'rm_sp': 131, 'good_spp': 6312}


### ans none and early ans

In [56]:
print(len(agent.cases['ans_none']), len(agent.cases['ans_none'] & agent.cases['early_ans']), len(agent.cases['early_ans']))

0 0 2


In [57]:
print(len(agent.cases['ans_none'] - agent.cases['good_spp']), len(agent.cases['ans_none'] & agent.cases['good_spp']), len(agent.cases['ans_none']))
print(len(agent.cases['ans_none'] - agent.cases['good_ans']), len(agent.cases['ans_none'] & agent.cases['good_ans']), len(agent.cases['ans_none']))

0 0 0
0 0 0


In [58]:
print(len(agent.cases['early_ans'] - agent.cases['good_spp']), len(agent.cases['early_ans'] & agent.cases['good_spp']), len(agent.cases['early_ans']))
print(len(agent.cases['early_ans'] - agent.cases['good_ans']), len(agent.cases['early_ans'] & agent.cases['good_ans']), len(agent.cases['early_ans']))

1 1 2
0 2 2


### false final sp sentences

In [59]:
print(len(agent.cases['bad_sp'] - agent.cases['good_spp']), len(agent.cases['bad_sp'] & agent.cases['good_spp']), len(agent.cases['bad_sp']))
print(len(agent.cases['bad_sp'] - agent.cases['good_ans']), len(agent.cases['bad_sp'] & agent.cases['good_ans']), len(agent.cases['bad_sp']))

0 2237 2237
654 1583 2237


### **false final sp passages**

In [60]:
bad_spp_cases = set(agent.observed.keys()) - agent.cases['good_spp']

In [61]:
print(len(bad_spp_cases - agent.cases['good_ans']), len(bad_spp_cases & agent.cases['good_ans']), len(bad_spp_cases))

804 289 1093


In [62]:
from collections import Counter

rec_spp_nums = [len(set(agent.pretty_memory(q_id).keys()) & set(gold_qas_map[q_id][2].keys())) for q_id in bad_spp_cases]

print(Counter(rec_spp_nums))

Counter({1: 809, 0: 265, 2: 19})


### **add distractor into memory**

In [63]:
print(len(agent.cases['add_dis'] - agent.cases['good_spp']), len(agent.cases['add_dis'] & agent.cases['good_spp']), len(agent.cases['add_dis']))
print(len(agent.cases['add_dis'] - agent.cases['good_ans']), len(agent.cases['add_dis'] & agent.cases['good_ans']), len(agent.cases['add_dis']))

874 187 1061
665 396 1061


In [129]:
agent.cases['add_dis'] & agent.cases['good_spp']

{'5a7150c75542994082a3e7be',
 '5a72125055429971e9dc9241',
 '5a72160e55429971e9dc9253',
 '5a721a7655429971e9dc9271',
 '5a7293655542991f9a20c4ff',
 '5a7294b65542991f9a20c509',
 '5a72e28f5542992359bc31ba',
 '5a730abe5542994cef4bc42f',
 '5a7322165542994cef4bc470',
 '5a7329a15542991f9a20c65f',
 '5a7509175542993748c897a3',
 '5a75176a5542993748c897b6',
 '5a754fc35542996c70cfaedc',
 '5a7553a95542996c70cfaedf',
 '5a75e814554299109176e5c7',
 '5a76394c5542994ccc918725',
 '5a772e465542994aec3b7209',
 '5a773d8955429966f1a36cc4',
 '5a774e415542994aec3b7280',
 '5a776a7055429966f1a36d32',
 '5a77731955429972597f1541',
 '5a778ce55542995d831811c0',
 '5a77c1505542997042120b1b',
 '5a78f1b555429970f5fffde9',
 '5a78f3d255429970f5fffdf1',
 '5a792ad055429907847277d1',
 '5a7949d45542994bb9456fdd',
 '5a79a26d554299029c4b5f63',
 '5a7a4f405542994f819ef1c0',
 '5a7a650355429941d65f2615',
 '5a7af32e55429931da12c99c',
 '5a7b57155542992d025e680f',
 '5a7ba11d554299294a54aa39',
 '5a7c67c15542990527d55498',
 '5a7cba005542

### **miss sp passage**

In [64]:
print(len(agent.cases['miss_sp'] - agent.cases['good_spp']), len(agent.cases['miss_sp'] & agent.cases['good_spp']), len(agent.cases['miss_sp']))
print(len(agent.cases['miss_sp'] - agent.cases['good_ans']), len(agent.cases['miss_sp'] & agent.cases['good_ans']), len(agent.cases['miss_sp']))

252 197 449
279 170 449


In [117]:
print(agent.cases['miss_sp'] & agent.cases['good_spp'] - agent.cases['miss_sp2'] )

{'5a792ad055429907847277d1', '5a8c06bb5542995e66a47530', '5ab7d33d5542992aa3b8c859', '5ae5fd395542996de7b71ab3', '5a72125055429971e9dc9241', '5ab91f98554299753720f70d', '5ac3caa15542995ef918c23a', '5ab5b6c1554299637185c5e5', '5a8760e75542996e4f3087af', '5a8f43555542992414482a21', '5ab9ba6e554299743d22ebcd', '5a89fc305542993b751ca9dc', '5ae724ec5542995703ce8bd8', '5a88dc5a5542993b751ca890', '5a7e0ed25542991319bc9424', '5a7b68525542997c3ec97151', '5adfcb5455429906c02daa48', '5a84b0705542991dd0999d86', '5ac00f795542996f0d89cb16', '5ade9c9355429975fa854f1b', '5abbfb5e554299114383a0d0', '5abcefc45542993a06baf9b4', '5a84d2775542994c784dda5f', '5ac4d314554299076e296e02', '5a8242c155429940e5e1a81d', '5a7a36e65542990198eaf0ea', '5a8318955542990548d0b177', '5a8ba60b554299240d9c205b', '5a7782b95542995d8318117f', '5a8b64695542997f31a41d04', '5ab61798554299488d4d9aaa', '5a72e28f5542992359bc31ba', '5ae30aa05542992decbdcdd7', '5a7bf0d75542996dd594b85c', '5ab8f53355429916710eb0d1', '5ade3d75554299728e

### miss 2nd sp passage

In [65]:
print(len(agent.cases['miss_sp2'] - agent.cases['good_spp']), len(agent.cases['miss_sp2'] & agent.cases['good_spp']), len(agent.cases['miss_sp2']))
print(len(agent.cases['miss_sp2'] - agent.cases['good_ans']), len(agent.cases['miss_sp2'] & agent.cases['good_ans']), len(agent.cases['miss_sp2']))

24 138 162
63 99 162


### **remove sp**

In [66]:
print(len(agent.cases['rm_sp'] - agent.cases['good_spp']), len(agent.cases['rm_sp'] & agent.cases['good_spp']), len(agent.cases['rm_sp']))
print(len(agent.cases['rm_sp'] - agent.cases['good_ans']), len(agent.cases['rm_sp'] & agent.cases['good_ans']), len(agent.cases['rm_sp']))

84 47 131
89 42 131


In [67]:
print(len(agent.cases['rm_sp'] & agent.cases['add_dis']), len(agent.cases['rm_sp'] - agent.cases['add_dis']), len(agent.cases['rm_sp']))
print(len(agent.cases['rm_sp'] & agent.cases['add_dis'] & agent.cases['good_spp']), len(agent.cases['rm_sp'] & agent.cases['add_dis'] - agent.cases['good_spp']))
print(len(agent.cases['rm_sp'] - agent.cases['add_dis'] & agent.cases['good_spp']), len(agent.cases['rm_sp'] - agent.cases['add_dis'] - agent.cases['good_spp']))

88 43 131
16 72
31 12


### **false dense query expansion**

In [68]:
print(len(agent.cases['false_expansion'] - agent.cases['good_spp']), len(agent.cases['false_expansion'] & agent.cases['good_spp']), len(agent.cases['false_expansion']))
print(len(agent.cases['false_expansion'] - agent.cases['good_ans']), len(agent.cases['false_expansion'] & agent.cases['good_ans']), len(agent.cases['false_expansion']))

311 126 437
287 150 437


In [69]:
print(len(agent.cases['false_expansion'] - agent.cases['add_dis']), len(agent.cases['false_expansion']), len(agent.cases['false_expansion'] & agent.cases['add_dis']), len(agent.cases['add_dis']), len(agent.cases['add_dis'] - agent.cases['false_expansion']))

0 437 437 1061 624


## final evidence set size

In [71]:
from collections import defaultdict

msz2cases = defaultdict(set)
for q_id, mem in agent.memory.items():
    msz2cases[len(mem)].add(q_id)

for msz in sorted(msz2cases.keys()):
    cases = msz2cases[msz]
    print(f"{msz}: {len(cases)}")
    print(len(cases - agent.cases['good_spp']), len(cases & agent.cases['good_spp']), len(cases - agent.cases['good_ans']), len(cases & agent.cases['good_ans']))
    print()

0: 12
12 0 12 0

1: 173
173 0 173 0

2: 7197
892 6305 2157 5040

3: 22
15 7 11 11

20: 1
1 0 1 0



## case studty

In [ ]:
for x in agent.exhausted_cmds.values():
    if len(x) > 0:
        print(x)

In [ ]:
cnt = 0
for k, v in agent.memory.items():
    if len(v) >= 3 and k in predictions['answer']:
        if cnt == 0:
            print(k, v)
        cnt += 1
print(cnt)

In [233]:
q_id = '5abee70a5542994516f45476'  # 5ab3abe9554299753aec597e 5a7363ec5542991f29ee2dd7 5ae54bbd5542990ba0bbb262 5adf29f05542993344016c09 5a764c0b55429976ec32bd89
if q_id in predictions['answer']:
    print(predictions['answer'][q_id])
print(agent.all_answer[q_id])
if q_id in predictions['sp']:
    print(predictions['sp'][q_id])
print(agent.all_sp_facts[q_id])
if q_id in predictions['spp']:
    print(predictions['spp'][q_id])
print(agent.pretty_memory(q_id))

Mistress of the Robes
{'mistress of robes': 2.1536680459976196}
[('Mistress of the Robes', 0), ('Anne Innes-Ker, Duchess of Roxburghe', 1), ('Anne Innes-Ker, Duchess of Roxburghe', 0)]
{('Anne Innes-Ker, Duchess of Roxburghe', 1): 0.9742681980133057, ('Mistress of the Robes', 0): 0.988511323928833, ('Anne Innes-Ker, Duchess of Roxburghe', 0): 0.5971236228942871}
['Mistress of the Robes', 'Anne Innes-Ker, Duchess of Roxburghe']
{'Anne Innes-Ker, Duchess of Roxburghe': 0.9852555990219116, 'Mistress of the Robes': 0.9879997372627258}


In [234]:
gold_qas_map[q_id]

('What role did Anne Innes-Ker, Duchess of Roxburghe (daughter of the 7th Duke of Marlborough) serve to Queen Victoria?',
 'Mistress of the Robes',
 {'Anne Innes-Ker, Duchess of Roxburghe': [0, 1],
  'John Spencer-Churchill, 7th Duke of Marlborough': [0]})

In [232]:
print(agent.pretty_behavior(q_id))

MDR(Q, 2) / LINK('Bobcat Stadium (Texas State)') / LINK('Texas State Bobcats football') / BM25('team that was coached by Barry Switzer in 1981') / LINK('1987 Oklahoma Sooners football team') / MDR(Q) / LINK('Falcon Stadium') / MDR(Q, 2) / LINK('Doyt Perry Stadium') / MDR(Q) / LINK('Gaylord Family Oklahoma Memorial Stadium') / ANSWER('86,112,')


In [229]:
print(f"*{0:<3d} Q\t{agent.pretty_cmd(agent.commands[q_id][0])}")
for i, (obs, obs_score, cmd, proposals) in enumerate(zip(agent.ids2titles(agent.observed[q_id]), agent.obs_scores[q_id], agent.commands[q_id][1:], agent.proposals[q_id][1:])):
    print(f"*{i + 1:<3d} {obs_score:.3f} {obs}\t{agent.pretty_cmd(cmd)}")

*0   Q	MDR(Q)
*1   0.018 1981 Florida State Seminoles football team	MDR(Q)
*2   0.016 1981 Southwest Texas State Bobcats football team	LINK('Bobcat Stadium (Texas State)')
*3   0.483 Bobcat Stadium (Texas State)	LINK('Texas State Bobcats football')
*4   0.025 Texas State Bobcats football	BM25('team that was coached by Barry Switzer in 1981')
*5   0.190 1988 Orange Bowl	LINK('1987 Oklahoma Sooners football team')
*6   0.015 1987 Oklahoma Sooners football team	MDR(Q)
*7   0.020 1981 Air Force Falcons football team	LINK('Falcon Stadium')
*8   0.013 Falcon Stadium	MDR(Q)
*9   0.011 1981 Bowling Green Falcons football team	MDR(Q)
*10  0.019 2001 Bowling Green Falcons football team	LINK('Doyt Perry Stadium')
*11  0.036 Doyt Perry Stadium	MDR(Q)
*12  0.973 1981 Oklahoma Sooners football team	LINK('Gaylord Family Oklahoma Memorial Stadium')
*13  0.983 Gaylord Family Oklahoma Memorial Stadium	ANSWER('86,112,')


In [215]:
print(f"*{0:<3d} Q    {agent.pretty_cmd(agent.commands[q_id][0])}")
for cmd, conf, prob in agent.proposals[q_id][0]:
    print(f"{prob:.3f}  {conf:.2f}  {agent.pretty_cmd(cmd)}")
for i, (obs, obs_score, cmd, proposals) in enumerate(zip(agent.ids2titles(agent.observed[q_id]), agent.obs_scores[q_id], agent.commands[q_id][1:], agent.proposals[q_id][1:])):
    print(f"*{i + 1:<3d} {obs_score:.3f} -- {obs}    {agent.pretty_cmd(cmd)}")
    for cmd, conf, prob in proposals:
        print(f"{prob:.3f}  {conf:.2f} {agent.pretty_cmd(cmd)}")

*0   Q    MDR(Q)
0.001  1.00  ANSWER('noanswer')
0.375  1.00  BM25('Darl Hall')
0.624  1.00  MDR(Q)
0.000  1.00  LINK('nolink')
*1   0.158 -- Hall &amp; Oates    MDR(Q)
0.001  1.00 ANSWER('noanswer')
0.265  1.00 BM25('John Oats')
0.734  0.84 MDR(Q)
0.000  0.99 LINK('nolink')
*2   0.010 -- Maneater (Hall &amp; Oates song)    BM25('Darl Hall and John Oats')
0.001  0.99 ANSWER('noanswer')
0.512  1.00 BM25('Darl Hall and John Oats')
0.487  1.00 MDR(Q)
0.000  0.82 LINK('nolink')
*3   0.009 -- Flahavan's    MDR(Q)
0.001  1.00 ANSWER('noanswer')
0.417  1.00 BM25('John Oats')
0.581  1.00 MDR(Q)
0.000  1.00 LINK('nolink')
*4   0.041 -- England Dan &amp; John Ford Coley    MDR(Q)
0.002  0.97 ANSWER('noanswer')
0.096  1.00 BM25('"I\'d Really Love to See You Tonight"')
0.901  0.97 MDR(Q)
0.001  0.78 LINK('nolink')
*5   0.011 -- Harmonica Fats    MDR(Q)
0.001  0.99 ANSWER('noanswer')
0.173  1.00 BM25('"Tore Up"')
0.825  1.00 MDR(Q)
0.001  1.00 LINK('nolink')
*6   0.009 -- Private Eyes (Hall &amp; O

In [ ]:
agent.clicked[q_id]

In [ ]:
agent.observed[q_id]

In [ ]:
agent.obs_scores[q_id]